In [0]:
import pickle
import pandas as pd
import numpy as np
from attention import AttentionLayer
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
with open('var.pickle','rb') as f:
  char_indices,eng_char_indices,to_ts,from_ts= pickle.load(f)

In [0]:
def value_to_key(search,word_index):
  for word, num in word_index.items():   
    if num == search:
        return word
      
def char_to_sequences(x,char_indices):
  for sen in x:
    seq = []
    for char in sen:
      seq.append(char_indices[char])    
    yield seq
def sequences_to_texts(texts, word_index):
    for tokens in texts:
        temp = []
        for w in tokens:
          temp.append(value_to_key(w,word_index))
        yield temp

In [3]:
encoder_model = load_model("encoder_model.h5")



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [4]:
decoder_model = load_model("decoder_model.h5",custom_objects={"AttentionLayer":AttentionLayer})

In [0]:

def predict(word):
  word = list(char_to_sequences([word],char_indices))
  word = pad_sequences(word, padding="pre", maxlen=from_ts)

  

  test_from = word[0]
  test_from = test_from.reshape(1,test_from.shape[0])
  
  test_to = list(char_to_sequences(["*"],eng_char_indices))
  test_to = np.array(test_to)
  test_to = test_to.reshape(1,test_to.shape[0])


  enc_outs, enc_fwd_state, enc_back_state = encoder_model.predict([test_from])
  dec_state = np.concatenate([enc_fwd_state, enc_back_state], axis=-1)
  attention_weights = []
  to_text = []

  for i in range(to_ts):
    
    dec_out, attention, dec_state = decoder_model.predict([enc_outs, dec_state, test_to])
    dec_ind = np.argmax(dec_out, axis=-1)[0, 0]
    end = eng_char_indices["*"]
    if dec_ind == end:
        break
    test_to = [[dec_ind]]
    test_to = np.array(test_to)
    test_to = test_to.reshape(1,test_to.shape[0])

    attention_weights.append((dec_ind, attention))
    to_text.append(dec_ind)
    result = list(sequences_to_texts([to_text],eng_char_indices))[0]
    predicted_res = str("".join(result)).strip()

  return predicted_res


In [9]:
predict("इन्साइक्लपीडीअ")

'insiclapidy'